In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.client import device_lib
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from pathlib import Path

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [4]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [5]:
img_width, img_height = 150,150
batch_size = 10
epochs = 100
COUNT_NORMAL_TRAIN = 1342
COUNT_PNEUMONIA_TRAIN = 3876
TRAIN_IMAGE_COUNT = COUNT_NORMAL_TRAIN + COUNT_PNEUMONIA_TRAIN
weight_for_0 = (1 / COUNT_NORMAL_TRAIN ) * (TRAIN_IMAGE_COUNT) / 2.0
weight_for_1 = (1 / COUNT_PNEUMONIA_TRAIN) * (TRAIN_IMAGE_COUNT) / 2.0
class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.94
Weight for class 1: 0.67


In [6]:
if K.image_data_format()=="channels_first":
    input_shape =(3,img_width, img_height)
else:
    input_shape =(img_width, img_height,3)


In [7]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='./chest_xray/chest_xray/train/',
    labels='inferred',
    seed=123,
    validation_split=0.2,
    subset='training',
    batch_size=batch_size,
    image_size=(img_height,img_width)
)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.


In [8]:
# validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     directory='./chest_xray/chest_xray/val/',
#     labels='inferred',
#     batch_size=batch_size,
#     image_size=(img_height,img_width)
# )
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='./chest_xray/chest_xray/train/',
    labels='inferred',
    seed=123,
    validation_split=0.2,
    subset='validation',
    batch_size=batch_size,
    image_size=(img_height,img_width)
)

Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


In [ ]:
class_names = train_dataset.class_names
class_names

In [ ]:
plt.figure(figsize=(10, 10))

for images,labels in train_dataset.take(8):
    for i in range (9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')


In [ ]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
train_ds.enumerate()

In [ ]:
model = Sequential()
# model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'))
# model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.3))
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255,input_shape=input_shape))
#   layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
model.add(Conv2D(32,(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.summary()

In [ ]:
model.compile(
    loss= "binary_crossentropy", 
    optimizer="adam",
    metrics=METRICS)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    batch_size=batch_size,
    class_weight=class_weight,
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='./chest_xray/chest_xray/test/',
    labels='inferred',
    batch_size=1,
#     seed=123,
#     subset='training',
#     validation_split=0.2
    image_size=(img_height,img_width)
)

In [ ]:
test_ds = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
model.evaluate(
    test_ds,
    batch_size=1,
)

In [ ]:
eval_pneumonia = tf.keras.preprocessing.image_dataset_from_directory(
    directory='./chest_xray/chest_xray/test_val/preumonia',
    labels='inferred',
    batch_size=1,
    image_size=(img_height,img_width)
)

In [ ]:
eval_pn = eval_pneumonia.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.evaluate(
    eval_pn,
    batch_size=1,
)

In [ ]:
eval_normal = tf.keras.preprocessing.image_dataset_from_directory(
    directory='./chest_xray/chest_xray/test_val/normal/',
    labels='inferred',
    batch_size=1,
    image_size=(img_height,img_width)
)

In [ ]:
eval_nr = eval_normal.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.evaluate(
    eval_nr,
    batch_size=1,
)